This ETL is for updating data into factrequestrfdcalculatedfields table. This ETL package is named as "factRequestsRFSCalcFields", to align with the pre-existing SSIS ETL for the same table.

In [0]:
# Install required packages
%restart_python 
%pip install maya holidays python-dateutil pytz
%pip install boto3




In [0]:
# Databricks notebook for datetimehandler class

import maya
from datetime import datetime, timedelta
from pytz import timezone

class datetimehandler:
    """Supports common date operations"""

    def gettoday(self, format=None):
        now_pst = maya.parse(maya.now()).datetime(to_timezone=self.getdefaulttimezone(), naive=False)
        return self.__formatdate(now_pst, format)

    def now(self):
        return maya.parse(maya.now()).datetime(to_timezone=self.getdefaulttimezone(), naive=False)
    
    def formatdate(self, input, format=None):
        _inpdate = self.getdate(input)
        return self.__formatdate(_inpdate, format) 

    def convert_to_pst(self, input, format=None):
        now_pst = maya.parse(self.getdate(input)).datetime(to_timezone=self.getdefaulttimezone(), naive=False)
        return self.__formatdate(now_pst, format) 

    def __formatdate(self, _inpdate, format):
        _format = format if format not in (None, '') else self.getdefaultdateformat()
        return _inpdate.strftime(_format)

    def getdate(self, inputdate):
        return datetime.strptime(inputdate, "%Y-%m-%d") if isinstance(inputdate, str) else inputdate            
   
    def getdefaulttimezone(self):
        return 'America/Vancouver'

    def getdefaultdateformat(self):
        return '%Y-%m-%d'

In [0]:
# Databricks notebook for duecalculator class

from os import stat
from re import VERBOSE
from datetime import datetime as datetime2
from datetime import datetime, timedelta
import holidays
import os
from dateutil.parser import parse
from pytz import timezone
# Use the datetimehandler class defined above in this notebook

class duecalculator:
    """ Due date calculator helper service """

    def getpreviousbusinessday(self, cfrduedate, ca_holidays):
        _prevbusinessday = self.__getpreviousweekday(cfrduedate)
        if not self.__isholiday(_prevbusinessday, ca_holidays):            
            return _prevbusinessday
        else:
            return self.getpreviousbusinessday(_prevbusinessday, ca_holidays)
        
    def getpreviousbusinessday_by_n(self, duedate, ca_holidays, n):
        _prevbusinessday = duedate
        for _ in range(n):
            _prevbusinessday = self.getpreviousbusinessday(_prevbusinessday, ca_holidays)
        return _prevbusinessday
    
    def formatduedate(self, input):
        return datetimehandler().formatdate(input)

    def addbusinessdays(self, inpdate, days):
        _holidays = self.getholidays()
        businessdays = 0
        __calcdate = datetimehandler().getdate(inpdate)
        while businessdays < days:
            __calcdate += timedelta(days=1)
            if self.isbusinessday(__calcdate, _holidays):
                businessdays += 1              
        return __calcdate

    def getbusinessdaysbetween(self, date1, date2=None):
        _holidays = self.getholidays()
        businessdays = 0
        date2 = date2 if date2 else self.gettoday()
        _date1_date_fmt = datetimehandler().getdate(date1)
        _date2_date_fmt = datetimehandler().getdate(date2)
        __fromdate = min(_date1_date_fmt, _date2_date_fmt)
        __todate = max(_date1_date_fmt, _date2_date_fmt)
        __fromcalcdate = __fromdate
        while __fromcalcdate.date() < __todate.date():
            if self.isbusinessday(__fromcalcdate, _holidays):
                businessdays += 1
            __fromcalcdate += timedelta(days=1)           
        return businessdays  

    def getholidays(self):
        ca_holidays = []
        currentyear = datetime.today().year
        years = [currentyear - 1, currentyear, currentyear + 1] 
        for year in years:
            ca_holidays.extend(self.__getholidaysbyyear(year))
        return ca_holidays

    def gettoday(self):
        return datetimehandler().gettoday()

    def now(self):
        return datetimehandler().now()

    def __getholidaysbyyear(self, year):        
        ca_holidays = []
        for date, name in sorted(holidays.CA(prov='BC', years=year).items()):
            ca_holidays.append(date.strftime(datetimehandler().getdefaultdateformat()))
        if 'FOI_ADDITIONAL_HOLIDAYS' in os.environ and os.getenv('FOI_ADDITIONAL_HOLIDAYS'):
            _addldays = os.getenv('FOI_ADDITIONAL_HOLIDAYS')
            for _addlday in _addldays.split(","):
                ca_holidays.append(_addlday.strip().replace('XXXX', str(year)))
        return ca_holidays    

    def isbusinessday(self, inpdate, holidays=None):
        _holidays = self.getholidays() if holidays is None else holidays
        if datetimehandler().formatdate(inpdate) not in _holidays and self.__isweekday(inpdate):
            return True
        return False

    def __isweekday(self, inpdate):
        _inpdate = datetimehandler().getdate(inpdate)
        return _inpdate.weekday() < 5
    
    def __getpreviousweekday(self, cfrduedate):
        _cfrduedate = datetimehandler().getdate(cfrduedate)  
        if _cfrduedate.weekday() == 0:
            diff = 3
        elif _cfrduedate.weekday() == 6:
            diff = 2
        else:
            diff = 1  
        res = _cfrduedate - timedelta(days=diff)
        return res.strftime(datetimehandler().getdefaultdateformat())           
   
    def __isholiday(self, input, ca_holidays):
        return input in ca_holidays

In [0]:

import boto3
import os
from botocore.exceptions import NoCredentialsError
from datetime import date
from pyspark.sql.functions import udf, col,to_date
from pyspark.sql.types import IntegerType, BooleanType
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers
 

def business_days_fromtoday(duedate):
    today = date.today().strftime("%Y-%m-%d")
    try:
        return duecalculator().getbusinessdaysbetween(duedate, today)
    except Exception:
        return None

def business_days_between(date1, date2):
    try:
        if date1 == None or date1 == 'NULL':
            return 0            
        result  = duecalculator().getbusinessdaysbetween(date1, date2)            
        return result   
    except Exception:
        return None    

try:
   
    tablename = "factrequestrfdcalculatedfields"
    runcycleid = etl_helpers.start_run_cycle(f"{tablename}")
    os.makedirs("/dbfs/foi/dataload", exist_ok=True)  # make sure directory exists

  

    df_lastrun = spark.sql(f"SELECT runcyclestartat as createddate FROM dimruncycle WHERE packagename = \"{tablename}\" AND success = 't' ORDER BY runcycleid DESC LIMIT 1")
        
    if df_lastrun.count() > 0:
        lastruntime = df_lastrun.first().createddate.strftime("%Y-%m-%d %H:%M:%S")
    else:
        lastruntime = "2019-01-01 00:00:00"
        print(lastruntime)

    query = f"""
        SELECT 
            frs.name as requeststatus,
            {runcycleid} as runcycleid,
            MIN(fmr.foirequest_id) AS foirequestid,
            MIN(fmr.startdate) AS startdate,
            MIN(fmr.duedate) AS duedate,
            MIN(fmr.cfrduedate) AS cfrduedate,           
            MIN(fmr.created_at) AS createdat,
            MIN(fmr.updated_at) AS updatedat,
            now() as todaydate
        FROM foi_mod.foiministryrequests fmr
            JOIN foi_mod.foirequeststatuses frs ON fmr.requeststatusid = frs.requeststatusid
            WHERE fmr.created_at > '{lastruntime}'
            GROUP BY frs.name
            ORDER BY MIN(fmr.created_at)
            """

    df = spark.sql(query)
    # df.show()

    df_mapped = df.selectExpr(
        "foirequestid AS foirequestid",
        "runcycleid AS runcycleid",
        "requeststatus as requeststatus",
        "startdate AS startdate",
        "duedate AS duedate",        
        "cfrduedate AS cfrduedate",
        "createdat AS createdat",
        "updatedat AS updatedat",
        "todaydate AS todaydate"
    )
       
    df_mapped.show()

    

    business_days_fromtodayudf = udf(business_days_fromtoday, IntegerType())
    business_days_between_udf = udf(business_days_between, IntegerType())


    df_transformed = df_mapped \
    .withColumn("overduedays", business_days_fromtodayudf(col("duedate")))\
    .withColumn("elapseddays", business_days_between_udf(col("startdate"), col("todaydate")))\
    .withColumn("passduedays", business_days_between_udf(col("duedate"), col("todaydate")))\
    .withColumn("remainingdays", business_days_between_udf(col("todaydate"), col("duedate")))\
    .withColumn("rfdage", business_days_between_udf(col("todaydate"), col("cfrduedate")))
   
   
  
    df_load_data =  df_transformed.select("foirequestid","runcycleid" , "overduedays", "elapseddays", "passduedays",  "rfdage","remainingdays","requeststatus")
    df_load_data.show()

    from delta.tables import DeltaTable
    delta_table = DeltaTable.forName(spark, f"hive_metastore.default.{tablename}")
    delta_table.alias("target").merge(
        df_load_data.alias("source"),
        "target.foirequestid = source.foirequestid"
    ).whenNotMatchedInsert(values = {
        "foirequestid": "source.foirequestid",
        "runcycleid": "source.runcycleid",
        "overduedays": "source.overduedays",
        "elapseddays": "source.elapseddays",
        "passduedays": "source.passduedays",
        "rfdage": "source.rfdage",
        "remainingdays": "source.remainingdays",
        "requeststatus":"source.requeststatus"
    }).execute()

    etl_helpers.end_run_cycle(runcycleid, 't', f"{tablename}")
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', f"{tablename}", "Credentials not available")
except Exception as e:
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', f"{tablename}")
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', f"{tablename}", f"An error occurred: {e}")